# Import and downloads of packages

In [ ]:
!python -m pip install pyyaml
import sys, os, distutils.core

In [ ]:
# Detectron 2
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Andre pakker

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
import pandas as pd

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
import os
import cv2
from detectron2.data import DatasetCatalog, MetadataCatalog
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import convert_to_coco_json
import shutil
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

In [ ]:
import csv
import torch
import math
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog

# Load model from GitHub

# Load images from github

# Initialise test images

In [ ]:
test_folder = "Dataprojekt/Data/Clean Data/Overbite Data/Annotated Verification Data" # Change folder for other images

def test_dataset_function():
    dataset_dicts = []
    for idx, filename in enumerate(os.listdir(test_folder)):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            record = {}
            file_path = os.path.join(test_folder, filename)

            height, width = cv2.imread(file_path).shape[:2]
            record["file_name"] = file_path
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width
            record["annotations"] = []  # Ingen annotations (ren test data)

            dataset_dicts.append(record)
    return dataset_dicts

DatasetCatalog.register("my_test_dataset", test_dataset_function)
MetadataCatalog.get("my_test_dataset").set(thing_classes=["object"])


# Option to remove dataset

dataset_name = "test_dataset"
if dataset_name in DatasetCatalog.list():
    DatasetCatalog.remove(dataset_name)
if dataset_name in MetadataCatalog.list():
    MetadataCatalog.pop(dataset_name)

# Initialise trained model

In [ ]:
import os
import torch
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from google.colab.patches import cv2_imshow

# Reinitialize config
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")) # Husk at bruge korrekt backbone
cfg.MODEL.WEIGHTS = os.path.join("model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Confidence threshold
cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1  # Match træning

predictor = DefaultPredictor(cfg)

print("Model reloaded successfully!")

# Run a test

In [ ]:
# File paths
labels_file = "Dataprojekt/Data/Clean Data/Overbite Data/Updated_Labels.csv"
image_folder = "Dataprojekt/Data/Clean Data/Overbite Data/Annotated Verification Data"
output_csv = "final_results_with_distance.csv"

# Conversion
PIXEL_TO_MM = 0.08

# Load metadata and test dataset
dataset_dicts = DatasetCatalog.get("my_test_dataset")
metadata = MetadataCatalog.get("my_test_dataset")

# === LOAD GROUND TRUTH LABELS ===
ground_truth = {}
with open(labels_file, mode="r", newline="") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        filename, x, y = [col.strip() for col in row]
        filename = os.path.splitext(filename)[0]  # remove extension
        ground_truth[filename] = (float(x), float(y))

# === MAIN LOOP: Predict, Match, Save, Visualize ===
rows = [["Filename", "X_True", "Y_True", "X_Model", "Y_Model", "Euc_dist", "mm_Dist"]]

for sample in dataset_dicts:
    img_path = sample["file_name"]
    filename = os.path.splitext(os.path.basename(img_path))[0]

    if filename not in ground_truth:
        print(f"⚠️ No ground truth for: {filename}")
        continue

    # Read image and run prediction
    img = cv2.imread(img_path)
    outputs = predictor(img)

    instances = outputs["instances"].to("cpu")
    if not instances.has("pred_keypoints"):
        print(f"❌ No keypoints found in: {filename}")
        continue

    keypoints = instances.pred_keypoints
    if len(keypoints) == 0:
        print(f"⚠️ Empty keypoint list in: {filename}")
        continue

    # Assume first instance
    pred_x, pred_y, _ = keypoints[0][0].tolist()
    true_x, true_y = ground_truth[filename]

    # Distance
    euc_dist = round(math.sqrt((true_x - pred_x) ** 2 + (true_y - pred_y) ** 2), 2)
    mm_dist = round(euc_dist * PIXEL_TO_MM, 2)

    # Save to CSV row
    rows.append([filename, true_x, true_y, pred_x, pred_y, euc_dist, mm_dist])

    # === VISUALIZE ===
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6, 6))
    plt.imshow(img_rgb)
    plt.scatter(true_x, true_y, color="blue", marker="o", s=3, label=f"True ({true_x}, {true_y})")
    plt.scatter(pred_x, pred_y, color="red", marker="x", s=3, label=f"Model ({pred_x}, {pred_y})")
    plt.legend(title=f"Dist: {euc_dist} px ({mm_dist} mm)")
    plt.title(f"Comparison: {filename}")
    plt.show()

# === SAVE FINAL CSV ===
with open(output_csv, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(rows)

print(f"✅ Final results saved to: {output_csv}")

# Distrubution of distances to True point

In [ ]:
import csv
import matplotlib.pyplot as plt

# File path
matched_predictions_file = "final_results_with_distance.csv"

# Read distance values
distances_mm = []
with open(matched_predictions_file, mode="r", newline="") as file:
    reader = csv.reader(file)
    next(reader)  # Skip first header
    next(reader)  # Skip second header

    for row in reader:
        distance_mm = float(row[-1])  # Last column is "Distance (mm)"
        distances_mm.append(distance_mm)

# Plot histogram
plt.figure(figsize=(8, 5))
plt.hist(distances_mm, bins=35, color="blue", edgecolor="black", alpha=0.7)

# Labels and title
plt.xlabel("Distance Error (mm)")
plt.ylabel("Frequency")
plt.title("Distribution of Prediction Errors (mm)")

# Show plot
plt.show()
